# MIE1621 Fall 2017 Project
## Jonathan Smith 999694521

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy as sci
from numpy.linalg import inv,norm
from scipy.optimize import minimize

### Data Initialization & Pre-Processing
Create functions that will convert a csv file into covariance and returns

In [2]:
### Manually input stock data
# stocks = pd.DataFrame(columns=['Return', 'var_1', 'var_2', 'var_3'],
#                    data=[[0.1073,0.02778,0.00387,0.00021],
#                         [0.0731,0.00387,0.01112,-0.0002],
#                         [0.0621,0.00021,-0.0002,0.00115]]
#                   )
# stocks.to_csv('stock_data.csv')

In [3]:
stocks = pd.read_csv('stock_data.csv')

In [4]:
def split (df):
    '''Function to split input dataframe into variance and return'''
    ret=df.loc[:,'Return'].as_matrix()
    var=df.iloc[:,2:].as_matrix()
    return ret, var

In [5]:
def kkt_convert (df,d=3.5):
    '''Function to split input dataframe into variance and return
    and convert them into kkt matrices'''
    ret=df.loc[:,'Return'].as_matrix()
    ret = np.append(ret,1)
    
    var=df.iloc[:,2:].as_matrix()
    var=d*var
    size=len(var)
    var = np.insert(var,size,1,axis=1)
    var = np.insert(var,size,1,axis=0)
    var[size][size]=0
    return ret, var 

In [6]:
ret_kkt, var_kkt = kkt_convert(stocks)

In [7]:
ret_kkt

array([ 0.1073,  0.0731,  0.0621,  1.    ])

In [8]:
var_kkt

array([[  9.72300000e-02,   1.35450000e-02,   7.35000000e-04,
          1.00000000e+00],
       [  1.35450000e-02,   3.89200000e-02,  -7.00000000e-04,
          1.00000000e+00],
       [  7.35000000e-04,  -7.00000000e-04,   4.02500000e-03,
          1.00000000e+00],
       [  1.00000000e+00,   1.00000000e+00,   1.00000000e+00,
          0.00000000e+00]])

In [9]:
ret, var = split(stocks)

In [10]:
ret

array([ 0.1073,  0.0731,  0.0621])

In [11]:
var

array([[ 0.02778,  0.00387,  0.00021],
       [ 0.00387,  0.01112, -0.0002 ],
       [ 0.00021, -0.0002 ,  0.00115]])

### Define Functions to Evaluate Objective Function & Derivatives

In [12]:
def f(x, d=3.5):
    '''Return value of function'''
#     return np.dot(x,ret)-np.dot(0.5*d*np.matmul(np.transpose(x),var),x)
    return np.dot(x,ret)-np.dot(0.5*d*np.matmul(x,var),x)

In [13]:
def df_kkt(x): 
    '''return value of kkt gradient'''
    return np.matmul(var_kkt,x)-ret_kkt

In [14]:
df_kkt([1,1,1,1])

array([ 1.00421 ,  0.978665,  0.94196 ,  2.      ])

In [15]:
# minimize(f,[1,1,1],method='BFGS')

In [16]:
# def d_f(x,d=4):
#     return ret-d*np.matmul(var,x)

In [17]:
# def d2_f(x,d=4):
#     return -d*var

In [18]:
f([1,1,1])

0.15883249999999999

### Define Functions for Newton, SDM, BFGS Methods

In [19]:
x=[1,1,1,1]
alpha=1
d=np.matmul(inv(var_kkt),df_kkt(x)) # From definition of newton
x_k = x - alpha*d
Rho=0.5
gamma=0.0001

print(f(x[:-1]))
print(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))

0.1588325
0.159215233116


In [20]:
def newton (df,d,init,tol=0.001,limit=100, backtrack=False, Rho=0.5, gamma=0.01):
    '''
    Perform Newton Method to find optimal solution for some function f
    df-dataframe containing return and covariance
    d-risk tolerance factor
    init-initial guess
    tol-stopping tolerance between interations
    limit-maximum number of iterations
    backtrack-perform backtrack line search if true, use step length 1 otherwise
    Rho,gamma-backtrack parameters
    '''
    ret,var=split(df)
    ret_kkt, var_kkt = kkt_convert(df)
    x=init
    rho=Rho
    delta=[tol]
    i=0
    print('Initial guess: ',x)
    while (np.any(np.absolute(delta)>=tol))&(i<limit):
        alpha=1
        d=np.matmul(inv(var_kkt),df_kkt(x)) # From definition of newton
        x_k = x - alpha*d
        while (alpha>0.01)&(backtrack==True)&(f(x_k[:-1])>(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))):
            alpha=alpha*rho
            x_k = x - alpha*d
        print('alpha: ',alpha)
        x_k = x - alpha*d
        delta = x_k-x
        x=x_k
        i+=1
        print('Iteration',i,':',x)
    print('slope:',df_kkt(x))
    x=x[0:-1]
    print('Solution: x =',x,', Return:',f(x))

In [38]:
def steepest_descent (df,d,init,tol=0.001,limit=100, backtrack=False, rho=0.5, gamma=0.01):
    '''
    Perform Steepest Descent Method to find optimal solution for some function f
    df-dataframe containing return and covariance
    d-risk tolerance factor
    init-initial guess
    tol-stopping tolerance between interations
    limit-maximum number of iterations
    backtrack-perform backtrack line search if true, use step length 1 otherwise
    Rho,gamma-backtrack parameters
    '''
    ret,var=split(df)
    ret_kkt, var_kkt = kkt_convert(df)
    x=init
    delta=[tol]
    i=0
    print('Initial guess: ',x)
#     while (np.any(np.absolute(delta)>=tol))&(np.all(df_kkt(x)<0))&(i<limit):
    while (np.any(np.absolute(delta)>=tol))&(i<limit):
        alpha=1
        d=df_kkt(x) # From definition of steepest descent
        x_k = x - alpha*d
        while (alpha>0.01)&(backtrack==True)&((f(x_k[:-1]))>(f(x[:-1])+alpha*gamma*np.matmul(d,(df_kkt(x))))):
            alpha=alpha*rho
            x_k = x - alpha*d
        print('alpha: ',alpha)
        x_k = x-alpha*d
        delta = x_k-x
        x=x_k
        i+=1
        print('Iteration',i,':',x)
        print('slope:',df_kkt(x))
    print('slope:',df_kkt(x))
    x=x[0:-1]
    print('Solution: x =',x,', Return:',f(x))

In [22]:
def BFGS (df,d,init,tol=0.001,limit=100, backtrack=False, rho=0.5, gamma=0.0001):
    '''
    Perform BFGS Quasi-Newton Method to find optimal solution for some function f
    '''
    ret,var=split(df)
    ret_kkt, var_kkt = kkt_convert(df)
    x=init
    H=inv(np.eye(1+len(var)))
    I=np.eye(1+len(var))
    delta=[tol]
    i=0
    print('Initial guess: ',x)
    while (np.any(np.absolute(delta)>=tol))&(i<limit):
        alpha=1
        d=np.matmul(np.asarray(H),df_kkt(x))
        x_k = x - alpha*d
        while (alpha>0.1)&(backtrack==True)&(f(x_k[:-1])>(f(x[:-1])+alpha*gamma*np.matmul(d,df_kkt(x)))):
            alpha=alpha*rho
            x_k = x - alpha*d
        print('alpha: ',alpha)
        x_k = x - alpha*d
        delta = x_k-x
#         find udpated H inverse
        s=np.asmatrix(x_k-x)
        y=np.asmatrix(df_kkt(x_k)-df_kkt(x))
        a=np.asscalar(1/(y*np.transpose(s)))
        A=a*(np.transpose(s)*y)
        B=a*(np.transpose(y)*s)
        C=a*(np.transpose(s)*(s))
        H=np.matmul(np.matmul((I-A),H),(I-B))+C 
        x=x_k
        i+=1
        print('Iteration',i,':',x)
    print('slope:',df_kkt(x))
    x=x[0:-1]
    print('Solution: x =',x,', Return:',f(x))

### Perform 3 Methods with Step Length = 1

In [40]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
newton(stocks,4,[1,1,1,1])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 4.05 µs
Initial guess:  [1, 1, 1, 1]
alpha:  1
Iteration 1 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
alpha:  1
Iteration 2 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
slope: [ 0.  0.  0.  0.]
Solution: x = [ 0.45526555  0.1745838   0.37015065] , Return: 0.0724980784211


In [41]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
# steepest_descent(stocks,4,[0.4,0.1,0.4,0.01])
steepest_descent(stocks,4,[1,1,1,1])

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.01 µs
Initial guess:  [1, 1, 1, 1]
alpha:  1
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
slope: [-1.1073777  -1.07236729 -1.06188442 -0.924835  ]
alpha:  1
Iteration 2 : [ 1.1031677   1.09370229  1.11992442 -0.075165  ]
slope: [-0.05956666 -0.09153965 -0.13271207  2.31679441]
alpha:  1
Iteration 3 : [ 1.16273436  1.18524194  1.25263649 -2.39195941]
slope: [-2.36923196 -2.4040574  -2.44899261  2.60061279]
alpha:  1
Iteration 4 : [ 3.53196632  3.58929934  3.70162909 -4.99257219]
slope: [-4.70512135 -4.88072732 -5.03968965  9.82289475]
alpha:  1
Iteration 5 : [  8.23708767   8.47002666   8.74131874 -14.81546694]
slope: [-14.00072353 -14.45346108 -14.8422579   24.44843308]
alpha:  1
Iteration 6 : [ 22.2378112   22.92348774  23.58357664 -39.26390002]
slope: [-36.88118507 -38.16011523 -39.23077777  67.74487558]
alpha:  1
Iteration 7 : [  59.11899627   61.08360297   62.81435441 -107.0087756 ]
slope: [-100.49438964 -103.

In [42]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
BFGS(stocks,4,[1,1,1,1])

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
Initial guess:  [1, 1, 1, 1]
alpha:  1
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
alpha:  1
Iteration 2 : [ 0.60147834  0.60766434  0.6395555  -0.5334302 ]
alpha:  1
Iteration 3 : [ 2.01458584  2.00211579  2.05697165 -1.36729661]
alpha:  1
Iteration 4 : [ 0.37926823  0.36520077  0.38977057  0.08642783]
alpha:  1
Iteration 5 : [ 0.33674282  0.31035651  0.33944003  0.05852694]
alpha:  1
Iteration 6 : [ 0.34756647  0.30907218  0.34196769  0.06184588]
alpha:  1
Iteration 7 : [ 0.37033821  0.29716183  0.34134922  0.06434301]
alpha:  1
Iteration 8 : [ 0.41764051  0.26367232  0.33493663  0.06539792]
alpha:  1
Iteration 9 : [ 0.45653346  0.22881726  0.32619087  0.06305928]
alpha:  1
Iteration 10 : [ 0.46733115  0.21358531  0.32220453  0.06030821]
alpha:  1
Iteration 11 : [ 0.46509935  0.21197629  0.32282827  0.0594258 ]
alpha:  1
Iteration 12 : [ 0.46282827  0.21104822  0.3247461   0.05912969]
alpha:  1
Iteration 13 :

### Perform 3 Methods with Backtracking

In [26]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
newton(stocks,4,[1,1,1,1], backtrack=True)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs
Initial guess:  [1, 1, 1, 1]
alpha:  1
Iteration 1 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
alpha:  0.03125
Iteration 2 : [ 0.45526555  0.1745838   0.37015065  0.06039773]
slope: [  8.32667268e-17  -2.22044605e-16  -1.17961196e-16  -2.66453526e-15]
Solution: x = [ 0.45526555  0.1745838   0.37015065] , Return: 0.0724980784211


In [27]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
steepest_descent(stocks,4,[1,1,1,1], backtrack=True, tol=0.0001)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs
Initial guess:  [1, 1, 1, 1]
slope: [ 1.00421   0.978665  0.94196   2.      ]
Solution: x = [1, 1, 1] , Return: 0.1588325


In [28]:
%time
ret, var = split(stocks)
ret_kkt, var_kkt = kkt_convert(stocks)
BFGS(stocks,4,[1,1,1,1], backtrack=True, tol=0.001)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.05 µs
Initial guess:  [1, 1, 1, 1]
alpha:  1
Iteration 1 : [-0.00421   0.021335  0.05804  -1.      ]
alpha:  0.0625
Iteration 2 : [ 0.03364552  0.05798058  0.09438472 -0.97083939]
alpha:  0.0625
Iteration 3 : [ 0.15745429  0.17948903  0.2170464  -0.99561796]
alpha:  1
Iteration 4 : [ 8.16394517  7.93417629  7.99447414  1.18320926]
alpha:  0.0625
Iteration 5 : [ 7.69597509  7.47886257  7.53735717  1.10615406]
alpha:  0.0625
Iteration 6 : [ 7.24234883  7.0365261   7.09340237  1.03963153]
alpha:  0.0625
Iteration 7 : [ 6.81304898  6.61710858  6.67268054  0.97826963]
alpha:  0.0625
Iteration 8 : [ 6.4099826   6.22258186  6.27717223  0.92093946]
alpha:  0.0625
Iteration 9 : [ 6.03227662  5.85218358  5.90609529  0.86722678]
alpha:  0.0625
Iteration 10 : [ 5.67849293  5.50458941  5.55810096  0.81687189]
alpha:  0.0625
Iteration 11 : [ 5.34715007  5.17842492  5.23179081  0.76965829]
alpha:  0.0625
Iteration 12 : [ 5.03683185  4.8723725 

### Scale up problem

In [29]:
# Read larger dataset into notebook
stocks_2 = pd.read_csv('stock_data_2.csv')
stocks_2

,Unnamed: 0,Return,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10
0,0,146.469681,0.011839,0.008182,0.009835,0.003280,0.007280,0.017415,-0.001371,0.007059,-0.013184,0.010996
1,1,903.057530,0.008182,0.007063,0.008208,0.003968,0.005458,0.013015,-0.000601,0.005886,-0.010116,0.008682
2,2,76.860558,0.009835,0.008208,0.011039,0.003828,0.007353,0.017705,-0.001448,0.006399,-0.012805,0.011075
3,3,94.038482,0.003280,0.003968,0.003828,0.034111,0.001837,0.004495,0.000287,0.004281,-0.005437,0.003056
4,4,283.091554,0.007280,0.005458,0.007353,0.001837,0.005797,0.013893,-0.001743,0.005104,-0.009706,0.007397
5,5,205.560718,0.017415,0.013015,0.017705,0.004495,0.013893,0.034623,-0.004600,0.013692,-0.024277,0.017413
6,6,103.744900,-0.001371,-0.000601,-0.001448,0.000287,-0.001743,-0.004600,0.001642,-0.001503,0.002834,-0.000858
7,7,2.426096,0.007059,0.005886,0.006399,0.004281,0.005104,0.013692,-0.001503,0.015276,-0.012211,0.005652
8,8,26.983626,-0.013184,-0.010116,-0.012805,-0.005437,-0.009706,-0.024277,0.002834,-0.012211,0.019165,-0.012049
9,9,835.000562,0.010996,0.008682,0.011075,0.003056,0.007397,0.017413,-0.000858,0.005652,-0.012049,0.013524


In [30]:
start=[1,1,1,1,1,1,1,1,1,1,1]

In [31]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
newton(stocks_2,4,start)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.29 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
alpha:  1
Iteration 1 : [ -1.85840171e+05   4.94761592e+05  -3.72246016e+05  -6.21134147e+03
   1.99877716e+05  -4.07846892e+04  -1.73250995e+05  -2.94058679e+04
   6.31942959e+03   1.06781342e+05   3.38085331e+01]
alpha:  1
Iteration 2 : [ -1.85840171e+05   4.94761592e+05  -3.72246016e+05  -6.21134147e+03
   1.99877716e+05  -4.07846892e+04  -1.73250995e+05  -2.94058679e+04
   6.31942959e+03   1.06781342e+05   3.38085331e+01]
slope: [  9.66338121e-13   1.59161573e-12   9.66338121e-13   7.10542736e-14
  -5.11590770e-13   1.70530257e-12   2.13162821e-13  -9.39692768e-13
   3.58824082e-13  -2.27373675e-13  -2.91038305e-11]
Solution: x = [-185840.17119016  494761.59180905 -372246.01588326   -6211.3414731
  199877.71647802  -40784.68923259 -173250.99466213  -29405.86793802
    6319.42959416  106781.34249804] , Return: 254935419.712


In [32]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
steepest_descent(stocks_2,4,start)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
slope: [-145.25502511 -901.88342478  -75.64639632  -92.85051419 -281.94220773
 -204.19890226 -102.77065732   -1.25237315  -26.25587978 -833.77345509
    9.        ]
Solution: x = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] , Return: 2676.53127075


In [33]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
BFGS(stocks_2,4,start)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
alpha:  1
Iteration 1 : [ 146.25502511  902.88342478   76.64639632   93.85051419  282.94220773
  205.19890226  103.77065732    2.25237315   27.25587978  834.77345509
   -8.        ]
alpha:  1
Iteration 2 : [  7.09360606e+04   4.44853315e+05   3.65518411e+04   4.55366983e+04
   1.38733363e+05   9.94564158e+04   5.09621794e+04   1.30731721e+02
   1.41628875e+04   4.10934534e+05  -3.27934806e+04]
alpha:  1
Iteration 3 : [ -251.70260394  1685.20213618  -430.08586731  -394.44908715    93.63416649
   -91.22359163  -377.61157189  -627.53175845  -590.62152395  1515.78460263
   566.18084732]
alpha:  1
Iteration 4 : [ -585.52727358  2950.24267248  -910.0802488   -743.77610305    68.16333293
  -344.51910368  -722.13702793 -1215.22651372 -1024.73137926  2609.33932008
   521.44114921]
alpha:  1
Iteration 5 : [ -7504.02019659  31380.46560303 -11057.61703999  -6797.67790113
    218.31

In [34]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
newton(stocks_2,4,start,limit=1000, backtrack=True, tol=0.01)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10.3 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
alpha:  0.0078125
Iteration 1 : [ -1.45088415e+03   3.86631712e+03  -2.90717981e+03  -4.75339178e+01
   1.56253685e+03  -3.17638197e+02  -1.35253121e+03  -2.28741156e+02
   5.03627312e+01   8.35221426e+02   1.25631667e+00]
alpha:  0.0078125
Iteration 2 : [ -2.89142545e+03   7.70143646e+03  -5.79263947e+03  -9.56886643e+01
   3.11187419e+03  -6.33787033e+02  -2.69548795e+03  -4.56687459e+02
   9.93398161e+01   1.66292550e+03   1.51063086e+00]
alpha:  0.0078125
Iteration 3 : [ -4.32071253e+03   1.15065939e+04  -8.65555647e+03  -1.43467202e+02
   4.64910733e+03  -9.47465957e+02  -4.02795285e+03  -6.82852931e+02
   1.47934267e+02   2.48416313e+03   1.76295822e+00]
alpha:  0.0078125
Iteration 4 : [ -5.73883330e+03   1.52820236e+04  -1.14961069e+04  -1.90872470e+02
   6.17433084e+03  -1.25869426e+03  -5.35000786e+03  -9.07251486e+02
   1.96149075e+02   3.29898484e+03   2.01331427

In [35]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
steepest_descent(stocks_2,4,start, backtrack=True, tol=0.01)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
slope: [-145.25502511 -901.88342478  -75.64639632  -92.85051419 -281.94220773
 -204.19890226 -102.77065732   -1.25237315  -26.25587978 -833.77345509
    9.        ]
Solution: x = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] , Return: 2676.53127075


In [36]:
%time
ret, var = split(stocks_2)
ret_kkt, var_kkt = kkt_convert(stocks_2)
BFGS(stocks_2,4,start, backtrack=True, tol=0.001)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
Initial guess:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
alpha:  0.0625
Iteration 1 : [ 10.07843907  57.36771405   5.72789977   6.80315714  18.62138798
  13.76243139   7.42316608   1.07827332   2.64099249  53.11084094   0.4375    ]
alpha:  1
Iteration 2 : [  7.09360606e+04   4.44853315e+05   3.65518411e+04   4.55366983e+04
   1.38733363e+05   9.94564158e+04   5.09621794e+04   1.30731721e+02
   1.41628875e+04   4.10934534e+05  -3.27934806e+04]
alpha:  0.0625
Iteration 3 : [  6.64873304e+04   4.17159222e+05   3.42406622e+04   4.26662712e+04
   1.30069501e+05   9.32354580e+04   4.77537576e+04   8.31968083e+01
   1.32407651e+04   3.85349469e+05  -3.07085577e+04]
alpha:  0.0625
Iteration 4 : [  6.22952100e+04   3.91271500e+05   3.20436367e+04   3.99530703e+04
   1.21944429e+05   8.73866640e+04   4.47239428e+04   1.91275799e+00
   1.23490758e+04   3.61428508e+05  -2.87566934e+04]
alpha:  0.0625
Iteration 5 : [  5.83450268e+04   3.6